## Log-playback (expert) trajectory extraction and usage

This notebook demonstrates how to extract expert actions using different dynamics models and step through scene with those actions. 

In [25]:
import os
from pathlib import Path
import mediapy

# Set working directory to the base directory 'gpudrive'
working_dir = Path.cwd()
while working_dir.name != 'gpudrive':
    working_dir = working_dir.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'gpudrive' not found")
os.chdir(working_dir)

from pygpudrive.env.config import EnvConfig, RenderConfig, SceneConfig
from pygpudrive.env.env_torch import GPUDriveTorchEnv

### Configurations

In [19]:
DYNAMICS_MODEL = "delta_local" # "delta_local" or "state"
DATA_PATH = "data/formatted_json_v2_no_tl_train"
MAX_NUM_OBJECTS = 128
NUM_ENVS = 4

# Configs
render_config = RenderConfig(draw_obj_idx=True)
scene_config = SceneConfig(path=DATA_PATH, num_scenes=NUM_ENVS)
env_config = EnvConfig(dynamics_model=DYNAMICS_MODEL)

### Make environment

In [20]:
env = GPUDriveTorchEnv(
    config=env_config,
    scene_config=scene_config,
    max_cont_agents=MAX_NUM_OBJECTS,
    device="cpu",
    render_config=render_config,
    action_type="continuous" # "continuous" or "discrete"
)


--- Ratio unique scenes / number of worls = 4 / 4 ---



### Get log-playback (expert) actions

- Different dynamics models have different action spaces. For details, [see the docs.](https://github.com/Emerge-Lab/gpudrive/tree/main/pygpudrive/env)

In [21]:
# Extract full expert trajectory
expert_actions, _, _ = env.get_expert_actions()

expert_actions.shape # Shape: (num_envs, num_steps, num_agents, action_space_dim)

torch.Size([4, 128, 91, 3])

### Step through an episode

In [22]:
# Reset environment
obs = env.reset()

frames = {f"env_{i}": [] for i in range(NUM_ENVS)}

# Step through the scene
for t in range(env_config.episode_len):

    env.step_dynamics(expert_actions[:, :, t, :])
    
    # Render the scenes
    for i in range(NUM_ENVS):
        frames[f"env_{i}"].append(env.render(i))

### Show resulting trajectories!

In [24]:
# Show videos
mediapy.show_videos(frames, fps=30, width=600, height=400, columns=2)

env_0 This browser does not support the video tag.,env_1 This browser does not support the video tag.
env_2 This browser does not support the video tag.,env_3 This browser does not support the video tag.
